In [ ]:
import tensorflow as tf
# Connect to TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.TPUStrategy(tpu)

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, MaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from google.colab import drive

In [ ]:
# Connect to Google Drive
drive.mount('/content/drive')

# Load the preprocessed and balanced dataset
df_features = pd.read_csv('/content/drive/MyDrive/yuksekTez/airline_dataset/preprocessed_balanced_features.csv')
df_labels = pd.read_csv('/content/drive/MyDrive/yuksekTez/airline_dataset/preprocessed_balanced_labels.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
X = df_features.values
y = df_labels['airline_sentiment'].values

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
input_dim = X_train.shape[1]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
# Create a simple CNN model
with strategy.scope():
    model = keras.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, output_dim=64, input_length=X_train.shape[1]),
        tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding="same", activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=64, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=3, activation="softmax")
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 12434, 64)         795776    
                                                                 
 conv1d_1 (Conv1D)           (None, 12434, 64)         12352     
                                                                 
 batch_normalization (BatchN  (None, 12434, 64)        256       
 ormalization)                                                   
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)               

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()

# Encode your string labels into integers
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert your data to TensorFlow tensors
X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train_encoded, dtype=tf.int32)
X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test_encoded, dtype=tf.int32)

from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
# Train the model using TensorFlow tensors and include early stopping
with strategy.scope():
    history = model.fit(X_train_tensor, y_train_tensor,
              epochs=100,
              batch_size=32,
              validation_data=(X_test_tensor, y_test_tensor),
              callbacks=[early_stopping])

Epoch 1/100
646/646 [==============================] - 28s 33ms/step - loss: 1.1284 - accuracy: 0.3403 - val_loss: 1.1142 - val_accuracy: 0.3308
Epoch 2/100
646/646 [==============================] - 13s 21ms/step - loss: 1.1048 - accuracy: 0.3391 - val_loss: 1.0990 - val_accuracy: 0.3300
Epoch 3/100
646/646 [==============================] - 13s 20ms/step - loss: 1.0987 - accuracy: 0.3409 - val_loss: 1.0998 - val_accuracy: 0.3308
Epoch 4/100
646/646 [==============================] - 13s 20ms/step - loss: 1.0970 - accuracy: 0.3419 - val_loss: 1.1736 - val_accuracy: 0.3308
Epoch 5/100
646/646 [==============================] - 13s 21ms/step - loss: 1.0961 - accuracy: 0.3398 - val_loss: 1.0964 - val_accuracy: 0.3308
Epoch 6/100
646/646 [==============================] - 13s 20ms/step - loss: 1.0958 - accuracy: 0.3405 - val_loss: 1.0954 - val_accuracy: 0.3300
Epoch 7/100
646/646 [==============================] - 13s 20ms/step - loss: 1.0951 - accuracy: 0.3376 - val_loss: 1.0934 - val_ac

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_tensor, y_test_tensor)
print('Test accuracy:', accuracy)

216/216 [==============================] - 3s 13ms/step - loss: 1.0924 - accuracy: 0.3598
Test accuracy: 0.35981985926628113


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# After training the model, you can make predictions
y_pred_probs = model.predict(X_test_tensor)

# Convert predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred_probs, axis=1)

# Define the class names corresponding to your sentiment classes
class_names = ["negative", "neutral", "positive"]

# Generate the classification report
classification_report_str = classification_report(y_test_tensor, y_pred_labels, target_names=class_names)
print('Classification Report:\n', classification_report_str)

# Generate the confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred_labels)

# Print or plot the confusion matrix as needed
print('Confusion Matrix:\n', confusion_mat)

216/216 [==============================] - 2s 8ms/step
Classification Report:
               precision    recall  f1-score   support

    negative       0.35      0.99      0.52      2374
     neutral       0.00      0.00      0.00      2233
    positive       0.55      0.05      0.10      2277

    accuracy                           0.36      6884
   macro avg       0.30      0.35      0.21      6884
weighted avg       0.30      0.36      0.21      6884



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ValueError: ignored

In [ ]:
!pip install matplotlib
import matplotlib.pyplot as plt

def plot_learning_curve(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

# Assuming 'history' is the variable storing your model's training history
plot_learning_curve(history)